# 1. Load libraries

In [1]:
import pandas as pd

# 2. Load data

In [2]:
df = pd.read_csv('data/feature.csv')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 23 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   user_id                                          956 non-null    object 
 1   days_passed_last_trip                            956 non-null    int64  
 2   days_passed_last_dropoff_trip                    956 non-null    float64
 3   days_passed_last_failed_trip                     956 non-null    float64
 4   days_passed_last_rider_cancel_trip               956 non-null    float64
 5   days_passed_last_driver_cancel_trip              956 non-null    float64
 6   numbers_of_dropoff_trips_last9m                  956 non-null    int64  
 7   numbers_of_failed_trips_last9m                   956 non-null    int64  
 8   numbers_of_trips_canceled_by_rider_last9m        956 non-null    int64  
 9   numbers_of_trips_canceled_by_dri

,user_id,days_passed_last_trip,days_passed_last_dropoff_trip,days_passed_last_failed_trip,days_passed_last_rider_cancel_trip,days_passed_last_driver_cancel_trip,numbers_of_dropoff_trips_last9m,numbers_of_failed_trips_last9m,numbers_of_trips_canceled_by_rider_last9m,numbers_of_trips_canceled_by_driver_last9m,...,total_distance_last9m,total_duration_last9m,avg_distance_last9m,avg_duration_last9m,avg_rider_score_last9m,is_last_trip_dropoff,is_last_trip_failed,is_last_trip_canceled_by_rider,is_last_trip_canceled_by_driver,label
0,00cb8ad6a0f7214d002e3ded6b7c9b80,122,122.0,0.0,0.0,0.0,7,0,0,0,...,88349.0,1723.0,12001.0,204.0,4.714286,1,0,0,0,1
1,00cb8ad6a0f7214d002e3ded6b7de012,100,100.0,0.0,296.0,0.0,5,0,2,0,...,42793.0,5722.0,8344.0,148.0,4.000000,1,0,0,0,1
2,00cb8ad6a0f7214d002e3ded6b7eb91b,316,316.0,0.0,0.0,0.0,1,0,0,0,...,15716.0,406.0,15716.0,406.0,5.000000,1,0,0,0,1
3,0194b4a5c9e41bfd35f9168423f7b7b4,314,0.0,0.0,314.0,0.0,0,0,2,0,...,0.0,0.0,0.0,0.0,4.663370,0,0,1,0,1
4,0194b4a5c9e41bfd35f9168423fa2857,136,136.0,0.0,174.0,0.0,23,0,11,0,...,125130.0,12231.0,5123.5,89.0,4.863636,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,ff1cb10b9edf79369742a37ce5d8d6f6,273,273.0,0.0,0.0,0.0,1,0,0,0,...,4776.0,76.0,4776.0,76.0,4.663370,1,0,0,0,1
952,ff5c924e0b630fd7c019a4234053c385,136,136.0,0.0,0.0,0.0,1,0,0,0,...,13720.0,61.0,13720.0,61.0,4.000000,1,0,0,0,1
953,ff5c924e0b630fd7c019a42340593ee2,259,259.0,0.0,0.0,0.0,1,0,0,0,...,3370.0,193.0,3370.0,193.0,5.000000,1,0,0,0,1
954,ff5c924e0b630fd7c019a42340594b55,92,92.0,0.0,111.0,112.0,30,0,7,1,...,224330.0,11363.0,4925.0,207.0,3.766667,1,0,0,0,1


# 3. Churn segmentation
Using the main idea of [Xia and He (2018)](https://www.atlantis-press.com/proceedings/mecae-18/25893766) and **RFM theory**. At the same time, regardless of the impact of the time node division on the customer's recent booking date, divide two variables of `F` and `M` into two states:
- **State 1**: F1 is lower than the average booking frequency, F2 is higher than the average booking frequency
- **State 2**: M1 is lower than the average of total spending, M2 is higher than the average of total spending

Recall from the modeling part, `numbers_of_dropoff_trips_last9m` and `total_spending_last9m` are top two important features to the model, we can consider `numbers_of_dropoff_trips_last9m` as `F` and `total_spending_last9m` as `M`.

In [14]:
# Select data and rename columns for analysis
seg_df = df.copy()
seg_df = seg_df[['numbers_of_dropoff_trips_last9m','total_spending_last9m']]\
         .rename(columns={'numbers_of_dropoff_trips_last9m':'F','total_spending_last9m':'M'})
# Rescale F and M into 5 bins
seg_df['F_quantile'] = pd.qcut(seg_df['F'], 5, ['1','2','3','4','5']).astype(int)
seg_df['M_quantile'] = pd.qcut(seg_df['M'], 5, ['1','2','3','4','5']).astype(int)
seg_df

,F,M,F_quantile,M_quantile
0,7,31587.0,4,4
1,5,17782.0,4,4
2,1,5368.0,1,2
3,0,0.0,1,1
4,23,50935.0,5,5
...,...,...,...,...
951,1,1700.0,1,1
952,1,4441.0,1,2
953,1,1700.0,1,1
954,30,73581.0,5,5


In [15]:
F_avg = seg_df['F_quantile'].mean()
M_avg = seg_df['M_quantile'].mean()

In [16]:
def segment(row):
  if row['F_quantile'] < F_avg and row['M_quantile'] < M_avg:
    cat = 'F1M1'
  elif row['F_quantile'] < F_avg and row['M_quantile'] > M_avg:
    cat = 'F1M2'
  elif row['F_quantile'] > F_avg and row['M_quantile'] < M_avg:
    cat = 'F2M1'
  else:
    cat = 'F2M2'
  return cat

In [17]:
seg_df['Churn_Segment'] = seg_df.apply(segment, axis = 1)

seg_df.info()
seg_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   F              956 non-null    int64  
 1   M              956 non-null    float64
 2   F_quantile     956 non-null    int32  
 3   M_quantile     956 non-null    int32  
 4   Churn_Segment  956 non-null    object 
dtypes: float64(1), int32(2), int64(1), object(1)
memory usage: 30.0+ KB


,F,M,F_quantile,M_quantile,Churn_Segment
0,7,31587.0,4,4,F2M2
1,5,17782.0,4,4,F2M2
2,1,5368.0,1,2,F1M1
3,0,0.0,1,1,F1M1
4,23,50935.0,5,5,F2M2
...,...,...,...,...,...
951,1,1700.0,1,1,F1M1
952,1,4441.0,1,2,F1M1
953,1,1700.0,1,1,F1M1
954,30,73581.0,5,5,F2M2


In [18]:
segment = pd.DataFrame(seg_df.Churn_Segment.value_counts().reset_index().values, columns=['Churn_Segment', 'Number_of_riders'])
segment = segment.sort_index(axis = 0, ascending=True)
segment['Rate(%)'] = 100*segment['Number_of_riders']/segment['Number_of_riders'].sum()
segment

,Churn_Segment,Number_of_riders,Rate(%)
0,F2M2,460,48.117155
1,F1M1,376,39.330544
2,F1M2,113,11.820084
3,F2M1,7,0.732218
